## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps 

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Salamiyah,SY,35.01,37.05,59.00,66,0,4.23,clear sky
1,1,New Norfolk,AU,-42.78,147.06,55.00,62,31,1.01,scattered clouds
2,2,Vardo,NO,70.37,31.11,41.00,75,75,28.86,broken clouds
3,3,Butaritari,KI,3.07,172.79,81.27,75,100,13.89,overcast clouds
4,4,Mataura,NZ,-46.19,168.86,61.86,42,54,5.75,broken clouds


In [3]:
city_data_df.loc[city_data_df["City"]=='Butaritari']

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
3,3,Butaritari,KI,3.07,172.79,81.27,75,100,13.89,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preffered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preffered_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
3,3,Butaritari,KI,3.07,172.79,81.27,75,100,13.89,overcast clouds
7,7,Manokwari,ID,-0.87,134.08,83.55,72,100,4.41,overcast clouds
12,12,Mount Isa,AU,-20.73,139.50,82.40,11,0,16.11,clear sky
17,17,Vaini,TO,-21.20,-175.20,78.80,74,40,8.05,scattered clouds
19,19,Cape Coast,GH,5.11,-1.25,78.89,89,76,10.47,broken clouds


In [13]:
# 4a. Determine if there are any empty rows.
preffered_cities_df.count()

City_ID        187
City           187
Country        185
Lat            187
Lng            187
Max Temp       187
Humidity       187
Cloudiness     187
Wind Speed     187
Description    187
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preffered_cities_df.dropna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
3,3,Butaritari,KI,3.07,172.79,81.27,75,100,13.89,overcast clouds
7,7,Manokwari,ID,-0.87,134.08,83.55,72,100,4.41,overcast clouds
12,12,Mount Isa,AU,-20.73,139.50,82.40,11,0,16.11,clear sky
17,17,Vaini,TO,-21.20,-175.20,78.80,74,40,8.05,scattered clouds
19,19,Cape Coast,GH,5.11,-1.25,78.89,89,76,10.47,broken clouds
...,...,...,...,...,...,...,...,...,...,...
668,668,Alofi,NU,-19.06,-169.92,78.80,65,75,12.75,broken clouds
671,671,Letlhakeng,BW,-24.09,25.03,75.40,28,5,7.94,clear sky
677,677,Sao Miguel Do Araguaia,BR,-13.28,-50.16,83.61,46,100,3.65,overcast clouds
679,679,Cayenne,GF,4.93,-52.33,77.00,100,0,11.12,clear sky


In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preffered_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)


,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Butaritari,KI,81.27,overcast clouds,3.07,172.79,
7,Manokwari,ID,83.55,overcast clouds,-0.87,134.08,
12,Mount Isa,AU,82.40,clear sky,-20.73,139.50,
17,Vaini,TO,78.80,scattered clouds,-21.20,-175.20,
19,Cape Coast,GH,78.89,broken clouds,5.11,-1.25,
24,Guerrero Negro,MX,76.21,scattered clouds,27.98,-114.06,
26,Carutapera,BR,79.93,moderate rain,-1.20,-46.02,
29,Atuona,PF,78.33,broken clouds,-9.80,-139.03,
32,Sao Filipe,CV,78.33,broken clouds,14.90,-24.50,
35,Merauke,ID,82.27,few clouds,-8.47,140.33,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df       

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Butaritari,KI,81.27,overcast clouds,3.07,172.79,Isles Sunset Lodge
7,Manokwari,ID,83.55,overcast clouds,-0.87,134.08,Swiss Belhotel Manokwari
12,Mount Isa,AU,82.40,clear sky,-20.73,139.50,ibis Styles Mt Isa Verona
17,Vaini,TO,78.80,scattered clouds,-21.20,-175.20,Keleti Beach Resort
19,Cape Coast,GH,78.89,broken clouds,5.11,-1.25,Samrit Hotel
...,...,...,...,...,...,...,...
668,Alofi,NU,78.80,broken clouds,-19.06,-169.92,Taloa Heights
671,Letlhakeng,BW,75.40,clear sky,-24.09,25.03,Tokey's Lodge
677,Sao Miguel Do Araguaia,BR,83.61,overcast clouds,-13.28,-50.16,Hotel Executivo Palace
679,Cayenne,GF,77.00,clear sky,4.93,-52.33,Hôtel Le Dronmi


In [19]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.loc[hotel_df["City"]=='Mount Isa']
hotel_df.dropna()
#hotel_df.isnull().sum()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Butaritari,KI,81.27,overcast clouds,3.07,172.79,Isles Sunset Lodge
7,Manokwari,ID,83.55,overcast clouds,-0.87,134.08,Swiss Belhotel Manokwari
12,Mount Isa,AU,82.40,clear sky,-20.73,139.50,ibis Styles Mt Isa Verona
17,Vaini,TO,78.80,scattered clouds,-21.20,-175.20,Keleti Beach Resort
19,Cape Coast,GH,78.89,broken clouds,5.11,-1.25,Samrit Hotel
...,...,...,...,...,...,...,...
668,Alofi,NU,78.80,broken clouds,-19.06,-169.92,Taloa Heights
671,Letlhakeng,BW,75.40,clear sky,-24.09,25.03,Tokey's Lodge
677,Sao Miguel Do Araguaia,BR,83.61,overcast clouds,-13.28,-50.16,Hotel Executivo Palace
679,Cayenne,GF,77.00,clear sky,4.93,-52.33,Hôtel Le Dronmi


In [27]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F </dd>  
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [29]:
import matplotlib.pyplot as plt

In [30]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig
#plt.savefig("Vacation_Search/WeatherPy_vacation_map.png")



Figure(layout=FigureLayout(height='420px'))